In [2]:
import numpy as np
import shap
import pandas as pd
from sklearn.linear_model import LogisticRegression
import sys
sys.path.append('../GilesCode/')
from helper import *
from helper2 import *
from helper2_dep import *
from helper2_indep import *
from helper2_shapley_sampling import *
from helper4_kshap import *
from os.path import join
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

/Users/jeremygoldwasser/opt/anaconda3/envs/shap/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Logistic dataset

In [3]:
# d = 10
# FEATURE_MEANS = np.repeat(0, d)
# FEATURE_VARS = np.repeat(1, d)
# FEATURE_COVS = [0.5, 0.25]
# COV_MAT = make_banded_cov(FEATURE_VARS, FEATURE_COVS)

# # Randomly generate samples
# np.random.seed(1)
# X = np.random.multivariate_normal(FEATURE_MEANS, COV_MAT, size=10000)
# X_train, X_test = X[:8000], X[8000:]
# xloc = X_test[10].reshape((1,d))

# np.random.seed(1)
# BETA = np.random.normal(0, 1, size = d)
# def model(x):
#     yhat = sigmoid(np.dot(x, BETA))
#     return yhat.item() if x.shape[0]==1 else yhat
# y = (model(X) > 0.5).astype(int)
# y_train, y_test = y[:8000], y[8000:]
# mapping_dict = None

### Census dataset

In [4]:
# X, y = shap.datasets.adult()
# X_display, y_display = shap.datasets.adult(display=True)
# X_binarized = pd.get_dummies(X_display)

# mapping_dict = {}
# for i, col in enumerate(X_display.columns):
#     bin_cols = []
#     for j, bin_col in enumerate(X_binarized.columns):
#         if bin_col.startswith(col):
#             bin_cols.append(j)
#     mapping_dict[i] = bin_cols

# X_norm = (X_binarized-X_binarized.min())/(X_binarized.max()-X_binarized.min())
# y_int = y_display.astype("int8")

# # Split into training and test sets
# np.random.seed(1)
# n, d = X_norm.shape
# n_train = round(n*0.75)
# train_idx = np.random.choice(n, size=n_train, replace=False)
# X_train_pd, y_train = X_norm.iloc[train_idx], y_int[train_idx]
# X_train = X_train_pd.to_numpy()

# test_idx = np.setdiff1d(list(range(n)), train_idx)
# X_test_pd, y_test = X_norm.iloc[test_idx], y_int[test_idx]
# X_test = X_test_pd.to_numpy()


### Bank dataset

In [5]:
# dirpath = "../Data/bank"
# # dirpath = /PATH/TO/DATA
# df_orig = pd.read_csv(join(dirpath, "df_orig.csv"))

# X_train_raw = np.load(join(dirpath, "X_train.npy"))
# X_test_raw = np.load(join(dirpath, "X_test.npy"))
# y_train = np.load(join(dirpath, "Y_train.npy"))
# y_test = np.load(join(dirpath, "Y_test.npy"))
# full_dim = X_train_raw.shape[1] # dimension including all binarized categorical columns
# X_df = pd.read_csv(join(dirpath, "X_df.csv"))


# trainmean, trainstd = np.mean(X_train_raw, axis=0), np.std(X_train_raw, axis=0)
# def rescale(x, trainmean, trainstd):
#     return (x - trainmean) / trainstd
# X_train = rescale(X_train_raw, trainmean, trainstd)
# X_test = rescale(X_test_raw, trainmean, trainstd)

# feature_means = np.mean(X_train, axis=0)
# cov_mat = np.cov(X_train, rowvar=False)


# df_orig.columns = df_orig.columns.str.replace(' ', '_')
# categorical_cols = ['Job', 'Marital', 'Education', 'Default', 'Housing',
#                     'Loan', 'Contact', 'Month', 'Prev_Outcome']
# mapping_dict = get_mapping_dict(df_orig, X_df, X_train_raw, categorical_cols)
# mapping_dict

# d = X_df.shape[1]


### German Credit 

In [6]:
# import sage
# df = sage.datasets.credit()
# # Property, other installment, housing, job, status of checking act, credit history, purpose, savings, employment since, marital status, old debtors
# n = df.shape[0]
# X_df = df.drop(["Good Customer"], axis=1)
# y = df["Good Customer"]

# categorical_columns = [
#     'Checking Status', 'Credit History', 'Purpose', #'Credit Amount', # It's listed but has 923 unique values
#     'Savings Account/Bonds', 'Employment Since', 'Personal Status',
#     'Debtors/Guarantors', 'Property Type', 'Other Installment Plans',
#     'Housing Ownership', 'Job', #'Telephone', 'Foreign Worker' # These are just binary
# ]
# X_binarized = pd.get_dummies(X_df, columns=categorical_columns)
# d_bin = X_binarized.shape[1]

# mapping_dict = {}
# for i, col in enumerate(X_df.columns):
#     bin_cols = []
#     for j, bin_col in enumerate(X_binarized.columns):
#         if bin_col.startswith(col):
#             bin_cols.append(j)
#     mapping_dict[i] = bin_cols

# np.random.seed(1)
# X_norm = (X_binarized-X_binarized.min())/(X_binarized.max()-X_binarized.min())
# n_train = round(n*0.8)
# train_idx = np.random.choice(n, n_train, replace=False)
# X_train, y_train = X_norm.iloc[train_idx].to_numpy(), y.iloc[train_idx].to_numpy()
# test_idx = np.setdiff1d(np.arange(n),train_idx)
# X_test, y_test = X_norm.iloc[test_idx].to_numpy(), y.iloc[test_idx].to_numpy()
# d = X_train.shape[1] # dimension of binarized data

### BRCA

In [7]:
# Load data
np.random.seed(1)
data = pd.read_csv('../Data/brca_small.csv')
X = data.values[:, :-1][:,:20]
Y = data.values[:, -1]
Y = (Y==2).astype(int)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=100, random_state=0)

# Normalize
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
# X_val = (X_val - mean) / std
X_test = (X_test - mean) / std
d = X_train.shape[1]
mapping_dict = None

In [8]:
sds = []
for i in range(d):
    uu = np.unique(X_train[:,i])
    if len(uu) == 2:
        sds.append(uu)
    else:
        sds.append(np.repeat(np.std(X_train[:,i]),2))
sds = np.array(sds)

feature_means = np.mean(X_train, axis=0)
cov_mat = np.cov(X_train, rowvar=False)
# Recondition covariance
u, s, vh = np.linalg.svd(cov_mat, full_matrices=True)
K = 10000
if np.max(s)/np.min(s) < K:
    cov2 = cov_mat
else:
    s_max = s[0]
    min_acceptable = s_max/K
    s2 = np.copy(s)
    s2[s <= min_acceptable] = min_acceptable
    cov2 = np.matmul(u, np.matmul(np.diag(s2), vh))


In [9]:
logreg = LogisticRegression().fit(X_train, y_train)
print("Class imbalance: {}".format(100*(max(np.mean(y_test), 1-np.mean(y_test)))))
print("Estimation accuracy: {}".format(np.mean((logreg.predict(X_test) > 0.5)==y_test)*100))


Class imbalance: 55.00000000000001
Estimation accuracy: 80.0


In [23]:
xloc = X_test[0:1]
def fmodel(xloc):
    return logreg.predict_proba(xloc)[:,1]

BETA = logreg.coef_.reshape(-1)
gradient = logreg_gradient(fmodel, xloc, BETA)
hessian = logreg_hessian(fmodel, xloc, BETA)

array([[ 4.54646748e-03, -3.23152384e-03, -5.73285701e-04,
         3.05522989e-03, -8.70348573e-03, -5.59504094e-03,
         5.02012330e-03, -6.99243644e-04, -5.06503693e-03,
         6.43905690e-03, -4.84344098e-04, -1.43364501e-04,
        -1.25581807e-03,  9.18165968e-04, -4.13602322e-03,
         2.95306350e-03,  1.43874720e-03,  4.48174441e-03,
        -6.24031159e-03, -5.03249254e-03],
       [-3.23152384e-03,  2.29689234e-03,  4.07478206e-04,
        -2.17158668e-03,  6.18623619e-03,  3.97682559e-03,
        -3.56818743e-03,  4.97006196e-04,  3.60011100e-03,
        -4.57673260e-03,  3.44260573e-04,  1.01900169e-04,
         8.92606415e-04, -6.52611116e-04,  2.93978957e-03,
        -2.09896918e-03, -1.02262820e-03, -3.18552019e-03,
         4.43546902e-03,  3.57697920e-03],
       [-5.73285701e-04,  4.07478206e-04,  7.22883200e-05,
        -3.85248464e-04,  1.09746390e-03,  7.05505315e-04,
        -6.33011216e-04,  8.81709559e-05,  6.38674589e-04,
        -8.11931301e-04,  6.1

In [81]:

    
gradient = difference_gradient(fmodel,xloc,sds)
hessian = difference_hessian(fmodel,xloc,sds)

shap_CV_true_indep = compute_true_shap_cv_indep(xloc, gradient, hessian, feature_means, cov_mat, mapping_dict=mapping_dict)
sum_shap_CV_true = np.sum(shap_CV_true_indep)
avg_CV_empirical = np.mean(f_second_order_approx(fmodel(xloc),X_train, xloc, gradient, hessian))
pred = fmodel(xloc)#[0]
exp_CV_sum_empirical = pred - avg_CV_empirical
print(sum_shap_CV_true)
print(exp_CV_sum_empirical)


-0.3885982815356193
[-0.38815471]


In [82]:
independent_features = True
obj_ss = cv_shapley_sampling(fmodel, X_train, xloc, 
                        independent_features,
                        gradient, hessian,
                        mapping_dict=mapping_dict,
                        M=100, n_samples_per_perm=10) # M is number of permutations
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_ss

order = np.argsort(np.abs(final_ests))[::-1]
print(np.round(100*(np.maximum(corr_ests, 0)**2)[order])) # Variance reductions


[36. 10. 41. 17. 38. 19.  0.  0. 11.  0. 40.  1. 11.  0.  3. 14.  2.  0.
  8.  0.]


In [83]:
independent_features = True
obj_kshap = cv_kshap(fmodel, X_train, xloc, 
            independent_features,
            gradient, hessian,
            mapping_dict=mapping_dict,var_method="wls",
            M=1000, n_samples_per_perm=10)
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_kshap

order = np.argsort(np.abs(final_ests))[::-1]
print(np.round(100*(np.maximum(corr_ests,0)**2)[order])) # Variance reductions

[16. 11. 13. 10. 13. 21. 18. 12. 15. 10. 11. 20. 15. 13. 16. 14. 11. 18.
 14. 16.]


In [84]:
# Prepare for dependent sampling
M_linear = 100
D_matrices = make_all_lundberg_matrices(M_linear, cov2)

In [85]:
independent_features = False
shap_CV_true_dep = linear_shap_vals(xloc, D_matrices, feature_means, gradient)
obj_dep = cv_shapley_sampling(fmodel, X_train, xloc,
                    independent_features,
                    gradient,
                    mapping_dict=mapping_dict,
                    shap_CV_true=shap_CV_true_dep, # Equivalently, can give D_matrices instead
                    M=100,n_samples_per_perm=10,
                    cov_mat=cov2)
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_dep
order = np.argsort(np.abs(final_ests))[::-1]
print(np.round(100*(np.maximum(corr_ests,0)**2)[order])) # Variance reductions

[27. 38. 14.  3.  0. 27. 18.  6. 25. 22. 35.  8. 11.  4. 10. 12. 10. 15.
 12. 27.]


Sometimes get negligible variance reductions from bootstrapped covariance, but good ones from WLS

In [86]:
# Sometimes get negligible variance reductions from bootstrapped covariance, but good ones from WLS
np.random.seed(2)
independent_features = False
obj_kshap_dep = cv_kshap(fmodel, X_train, xloc,
                    independent_features,
                    gradient,
                    mapping_dict=mapping_dict,
                    shap_CV_true=shap_CV_true_dep,
                    M=1000,n_samples_per_perm=10, var_method="wls",
                    cov_mat=cov2)
final_ests, vshap_ests_model, vshap_ests_CV, corr_ests = obj_kshap_dep
order = np.argsort(np.abs(final_ests))[::-1]
print(np.round(100*(np.maximum(corr_ests,0)**2)[order])) # Variance reductions

[ 6. 19. 10. 36. 18. 18. 23. 18. 23. 20. 24. 15. 31. 21. 22. 24. 24. 22.
 21. 18.]
